In [ ]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/2022-01_데이터마이닝이론및응용/0_텀프로젝트/데이터/버스노선/20220420기준_서울시_버스노선별정류소정보.csv")

In [ ]:
df.head()

,노선ID,노선명,순번,NODE_ID,ARS-ID,정류소명,X좌표,Y좌표
0,100100124,0017,1,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102000212,3306,산천동,126.953984,37.535420


In [ ]:
df = df.iloc[:2, :]

In [ ]:
df

,노선ID,노선명,순번,NODE_ID,ARS-ID,정류소명,X좌표,Y좌표
0,100100124,0017,1,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102000204,3298,청암동강변삼성아파트,126.949304,37.533961


In [ ]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[0]
        json_name = json_doc.get('address_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code


In [ ]:
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가


def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code # 전처리 함수에서 행정구역코드 리스트 받아서 데이터프레임에 추가

In [ ]:
APP_KEY = 'd7116f61fe9e88ae94f2649ac0e150c5' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

In [ ]:
for i in range(len(df)): 
    x_crd = float(df.loc[i, ['X좌표']])
    y_crd = float(df.loc[i, ['Y좌표']])
    address = get_address(x_crd, y_crd)
    df.loc[i, ['ADDRESS']] = address

    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

0 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [ ]:
#df저장
df.to_csv('busaddress.csv')